First, we import Google Drive and load the modules we'll need:

In [1]:
!pip install lyricsgenius

In [ ]:
from lyricsgenius import Genius
import json
import csv
import multiprocessing

# OS agnostic
import os 
CSV_PATH = os.path.join(os.path.curdir, 'artists', '10000-MTV-Music-Artists-page-%s.csv')

Now we set up the access token and import the dataset:

In [4]:
# Pull out artists

artists = []
for x in range(1,5):
  path = CSV_PATH % str(x)
  with open(path, encoding="UTF-8") as csvfile:
      TopArtists = csv.reader(csvfile)
      next(TopArtists)
      for row in TopArtists:
          artists.append(row[0])

print(len(artists))

8344


In [ ]:
# Genius setup
token = "EBufquOcw_ts4Y4V7yiddUNyUakTdqCpnMZhiI3XtAScWOntEom8Hj4T87gAV_cA"
genius = Genius(token, retries=2)

genius.verbose = False
genius.remove_section_headers = True
genius.skip_non_songs = True
genius.excluded_terms = ["(Remix)", "(Live)"]

process_number = int(multiprocessing.cpu_count())

shortened_list = artists[0:32]

temp_artists = multiprocessing.Manager().list(shortened_list)


# Processing
def clean_data(data):
    cleaned_data = data.replace("\n", " ").replace(",", " ")
    return cleaned_data

def process_artist(artist):
    artist_dict = artist.to_dict()
    return ""

def process_song(song):
    lyrics = clean_data(song.lyrics)
    return lyrics


# Run genius search
def search_genius(sub_list):
    try:
        sub_song_lyrics = []
        for artist in sub_list:
            print("Searching {artist}\n".format(artist=artist.strip()), end='')
            genius_artist = genius.search_artist(artist, per_page=50, get_full_info=False)
            if genius_artist == None:
                print("{artist} not found\n".format(artist=artist.strip()), end='')
                continue
            artist_data =  process_artist(genius_artist)
            print("{artist} number of songs: {song_num}\n".format(artist=artist.strip(), song_num=len(genius_artist.songs)), end='')
            for song in genius_artist.songs:
                song_data = process_song(song)
                sub_song_lyrics.append([song.title + " - " + song.artist, song.lyrics])
        return sub_song_lyrics
    except Error:
        print("Oops\n", end='')
    
    
def multi_processed():
    print("Multiprocessing with {process_number} processes\n".format(process_number=process_number))
    def split(a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

    split_list = list(split(shortened_list, process_number))
    # creating processes
    pool = multiprocessing.Pool(process_number)
    results = pool.map(search_genius, split_list)
    print(results)

def single():
    result = search_genius(shortened_list)
    print(result)
    
multi_processed()





Multiprocessing with 12 processes

